In [1]:
# torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소
# PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass)

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습. 
# torch.cuda 또는 torch.backends.mps 가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [14]:
# 신경망 모델을 nn.Module 의 하위클래스로 정의
# __init__ 에서 신경망 계층들을 초기화
# nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [15]:
# NeuralNetwork 의 인스턴스(instance)를 생성하고 
# 이를 device 로 이동한 뒤, 구조(structure)를 출력

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [32]:
# 모델을 사용하기 위해 입력 데이터를 전달
# -> 이는 일부 백그라운드 연산들과 함께 모델의 forward 를 실행
# -> model.forward() 를 직접 호출 금지!

# 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환
# 2차원 텐서의 
# ? dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, 
# dim=1에는 각 출력의 개별 값들이 해당
# 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 획득

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax()

print(f"The original value of class: {logits}")
print(f"The softmax value of class: {pred_probab}")
print(f"Predicted class: {y_pred}")

The original value of class: tensor([[-0.0775, -0.1220,  0.0350,  0.0266,  0.0050, -0.0559, -0.0050,  0.0205,
         -0.0062, -0.0120]], grad_fn=<AddmmBackward0>)
The softmax value of class: tensor([[0.0942, 0.0901, 0.1054, 0.1046, 0.1023, 0.0963, 0.1013, 0.1039, 0.1012,
         0.1006]], grad_fn=<SoftmaxBackward0>)
Predicted class: 2


In [39]:
# nn.Flatten 계층을 초기화하여 
# 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
# dim=0의 미니배치 차원은 유지

input_image = torch.rand(3, 28, 28)
print(input_image.size())

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

# torch.Size([3, 28, 28])
# torch.Size([3, 784])

torch.Size([3, 28, 28])
torch.Size([3, 784])


In [40]:
# nn.Linear
# 선형 계층은 저장된 가중치(weight)와 편향(bias)을 사용
# 입력에 선형 변환(linear transformation)을 적용하는 모듈

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [41]:
# nn.ReLU
# 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듦
# 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 
# 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.
# 이 모델에서는 nn.ReLU 를 선형 계층들 사이에 사용하지만, 
# 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입 가능

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0164,  0.1638,  0.6039, -0.0197, -0.3508,  0.5319, -0.2057, -0.1183,
         -0.0536, -0.2349, -0.1013,  0.1310,  0.3289,  0.2568,  0.4576,  0.4398,
          0.1183,  0.0032,  0.5397,  0.3530],
        [ 0.2285, -0.2444,  0.8031, -0.0763, -0.6805, -0.1081, -0.2019, -0.1925,
          0.2095, -0.3478, -0.0586, -0.1980,  0.0715,  0.5145,  0.4676,  0.5518,
          0.3135, -0.0285,  0.6864,  0.5769],
        [-0.2533, -0.1182,  0.6992,  0.1528, -0.2307,  0.2978, -0.3812, -0.1322,
          0.2623, -0.6546, -0.5436, -0.0583,  0.0264,  0.3378,  0.8147,  0.6610,
          0.3413,  0.0817,  0.8187,  0.4087]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0164, 0.1638, 0.6039, 0.0000, 0.0000, 0.5319, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1310, 0.3289, 0.2568, 0.4576, 0.4398, 0.1183, 0.0032,
         0.5397, 0.3530],
        [0.2285, 0.0000, 0.8031, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2095,
         0.0000, 0.0000, 0.0000, 0.0715, 0.5145, 0.46

In [42]:
# nn.Sequential
# nn.Sequential 은 순서를 갖는 모듈의 컨테이너
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달
# 순차 컨테이너(sequential container)를 사용하여 
# 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [44]:
# nn.Softmax
# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 
# 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환
# logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 
# [0, 1] 범위로 비례하여 조정(scale)
# dim 매개변수는 값의 합이 1이 되는 차원

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)


tensor([[0.0817, 0.1212, 0.0803, 0.0837, 0.0952, 0.1024, 0.1470, 0.0715, 0.0845,
         0.1324],
        [0.0880, 0.1265, 0.0825, 0.0878, 0.0845, 0.1127, 0.1387, 0.0700, 0.0828,
         0.1265],
        [0.0889, 0.1230, 0.0815, 0.0831, 0.0979, 0.0964, 0.1512, 0.0710, 0.0915,
         0.1154]], grad_fn=<SoftmaxBackward0>)


In [67]:
# Model's parameter
# 신경망 내부의 많은 계층들은 매개변수화(parameterize) 됨
# 즉, 학습 중에 최적화되는 가중치와 편향과 연관
# nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)
# 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됨

# 각 layer의 parameter를 설명
# Example)
# Layer: linear_relu_stack.0.weight
# Size: torch.Size([512, 784]) (output, input)
# Value: tensor([[-0.0303, -0.0045... (weight's value) -> 각 output 만큼 Input의 w 가 할당
# bias 는 각 output 에 하나(512), weight 는 각 output 별 input 갯수만큼 존재 (784*512)

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0148, -0.0179,  0.0294,  ..., -0.0211, -0.0301, -0.0206],
        [-0.0212,  0.0044,  0.0177,  ...,  0.0029, -0.0105, -0.0279]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0153,  0.0290], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0003, -0.0425, -0.0145,  ...,  0.0092, -0.0337,  0.0359],
        [-0.0033, -0.0391,  0.0421,  ...,  0.0265,  0.0035, -0.0441]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Size: 